In [1]:
#!git clone https://github.com/atcoder/ac-library.git

In [2]:
import re
import sys
from time import sleep
from pprint import pprint
from pathlib import Path
from functools import partial
import subprocess
from subprocess import PIPE

import numpy as np
import optuna
import pandas as pd
import matplotlib.pyplot as plt

FILENAME = "main.cpp"

In [3]:
!cp ../answer/{FILENAME} ./{FILENAME}

In [4]:
!clang++ {FILENAME} -std=gnu++17 -Wall -Wextra -O2 -o {FILENAME}.out -I/

In [5]:
SEED = 0
N_TESTCASES = 500

In [6]:
!seq {SEED} {SEED+N_TESTCASES-1} > seeds.txt && cargo run --release --manifest-path ../tools/Cargo.toml --bin gen seeds.txt
!ls in

    Updating crates.io index
  Downloaded quote v0.6.13                                                    m       Fetch [=======>                 ]  34.68%, 17.60MiB/s                  
  Downloaded syn v0.15.44                                                  
  Downloaded rand_chacha v0.2.2                                            
  Downloaded rand_pcg v0.2.1ng bytes: 182.6 KB                             
  Downloaded rand_core v0.5.1g bytes: 141.7 KB                             
  Downloaded rand_distr v0.2.2 bytes: 147.9 KB                             
  Downloaded unicode-xid v0.1.0bytes: 123.1 KB                             
  Downloaded proc-macro2 v0.4.30ytes: 109.7 KB                             
  Downloaded ppv-lite86 v0.2.10ytes: 60.6 KB                               
  Downloaded getrandom v0.1.16bytes: 44.3 KB                               
  Downloaded rand v0.7.3ining bytes: 11.5 KB                               
  Downloaded lazy_static v1.4.0                      

In [7]:
!cargo build --release --manifest-path ../tools/Cargo.toml --bin tester

   Compiling tools v1.0.0 (/home/user/ahc_httf_2022_qual/tools)
    Finished release [optimized] target(s) in 1.19ser(bin)                 


In [8]:
%%time
# もとのスコアの確認
scores = []
for i in range(N_TESTCASES):
    i = f"{i:04d}"
    score = !../tools/target/release/tester $(pwd)/{FILENAME}.out < in/{i}.txt 2>&1 | grep Score
    print(i, *score)
    score = int(score[0].split()[-1])
    scores.append(score)
np.mean(scores), np.mean(scores) * 50, np.std(scores)

0000 Score = 2236
0001 Score = 2122
0002 Score = 2484
0003 Score = 2102
0004 Score = 1867
0005 Score = 1660
0006 Score = 2246
0007 Score = 1975
0008 Score = 2426
0009 Score = 2309
0010 Score = 2373
0011 Score = 1966
0012 Score = 2148
0013 Score = 2157
0014 Score = 1995
0015 Score = 2038
0016 Score = 1986
0017 Score = 2291
0018 Score = 2076
0019 Score = 1640
0020 Score = 2337
0021 Score = 2338
0022 Score = 2321
0023 Score = 2001
0024 Score = 2208
0025 Score = 2308
0026 Score = 1590
0027 Score = 2361
0028 Score = 2259
0029 Score = 1970
0030 Score = 1940
0031 Score = 2096
0032 Score = 1587
0033 Score = 2133
0034 Score = 1961
0035 Score = 2441
0036 Score = 1743
0037 Score = 2335
0038 Score = 2289
0039 Score = 1844
0040 Score = 1831
0041 Score = 2248
0042 Score = 1625
0043 Score = 1958
0044 Score = 2182
0045 Score = 2073
0046 Score = 1983
0047 Score = 2163
0048 Score = 2317
0049 Score = 2159
0050 Score = 1764
0051 Score = 2245
0052 Score = 2240
0053 Score = 2163
0054 Score = 2215
0055 Score

0456 Score = 1703
0457 Score = 1426
0458 Score = 1895
0459 Score = 2224
0460 Score = 2143
0461 Score = 2063
0462 Score = 2310
0463 Score = 2131
0464 Score = 1896
0465 Score = 1789
0466 Score = 1964
0467 Score = 2377
0468 Score = 2075
0469 Score = 2058
0470 Score = 2075
0471 Score = 1956
0472 Score = 2023
0473 Score = 1650
0474 Score = 2216
0475 Score = 2221
0476 Score = 1762
0477 Score = 2256
0478 Score = 2345
0479 Score = 2368
0480 Score = 1601
0481 Score = 1706
0482 Score = 2026
0483 Score = 2235
0484 Score = 1604
0485 Score = 2041
0486 Score = 2310
0487 Score = 1700
0488 Score = 2056
0489 Score = 2172
0490 Score = 1902
0491 Score = 2037
0492 Score = 1681
0493 Score = 2241
0494 Score = 1994
0495 Score = 2324
0496 Score = 2188
0497 Score = 2148
0498 Score = 1819
0499 Score = 2111
CPU times: user 1.2 s, sys: 2.76 s, total: 3.95 s
Wall time: 18min 30s


(2012.298, 100614.9, 243.70076978951053)

In [6]:
# ここでエラー出力は何かおかしいかもしれない


# パラメータ抽出
with open(FILENAME) as f:
    answer = f.read()


variables_optimize = []

for left, value, right, options in re.findall(r"^([^/\n]*=\s*)(.+?)(\s*;\s*//(?:.*\W)?OPTIMIZE(\W.*))$", answer, re.MULTILINE):
    name = left.replace("=", "").strip().split()[-1]
    
    searched = re.search(r".*\[(?P<min>.*),(?P<max>.*)\].*", options)
    if searched:
        min_value = max_value = None
        try:
            min_value = eval(searched.group("min"))
            max_value = eval(searched.group("max"))
            assert min_value <= max_value
        except Exception as e:
            print(f"searched={searched}", file=sys.stderr)
            print(e, file=sys.stderr)
            continue
        log = "LOG" in options  # 雑、直したほうが良い
        if type(min_value) != type(max_value):
            print(f"searched={searched}", file=sys.stderr)
            print("types not matched", file=sys.stderr)
            continue
        if isinstance(min_value, int):
            method = "suggest_int"
        elif isinstance(min_value, float):
            method = "suggest_float"
        else:
            print(f"searched={searched}", file=sys.stderr)
            print(f"unknown type ({type(min_value)})", file=sys.stderr)
            continue
        variables_optimize.append({
            "name": name,
            "method": method,
            "min": min_value,
            "max": max_value,
            "log": log,
            "left": left,
            "right": right,
        })
    elif searched := re.search(r".*\{(?P<choices>.*?)\}.*", options):
        choices = list(map(lambda x: x.strip(), searched.group("choices").split(",")))
        variables_optimize.append({
            "name": name,
            "method": "suggest_categorical",
            "choices": choices,
            "left": left,
            "right": right,
        })
    else:
        print(f"searched={searched}", file=sys.stderr)
        print(f"pattern was matched but options are incorrect.", file=sys.stderr)

print(len(variables_optimize), "variables were found.")
if globals().get("pd"):
    display(pd.DataFrame(variables_optimize))
else:
    pprint(variables_optimize)



6 variables were found.


,name,method,min,max,log,left,right
0,MCMC_Q_L2_NORM_RANGE,suggest_float,2.000,20.0,True,constexpr auto MCMC_Q_L2_NORM_RANGE =,"; // OPTIMIZE LOG [2.0, 20.0]"
1,MCMC_Q_RANGE,suggest_float,0.200,10.0,True,constexpr auto MCMC_Q_RANGE =,"; // OPTIMIZE LOG [0.2, 10.0]"
2,EXPECTED_SKILL_EMA_ALPHA_COEF,suggest_float,0.020,2.0,True,constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF =,"; // OPTIMIZE LOG [0.02, 2.0]"
3,MAX_N_NOT_OPEN_TASKS_IN_QUEUE,suggest_int,60.000,100.0,False,constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE =,"; // OPTIMIZE [60, 100]"
4,PRIORITY_DAY_OFFSET,suggest_int,400.000,1200.0,False,constexpr auto PRIORITY_DAY_OFFSET =,"; // OPTIMIZE [400, 1200]"
5,PRIORITY_COEF,suggest_float,0.002,2.0,True,constexpr auto PRIORITY_COEF =,"; // OPTIMIZE LOG [0.002, 2.0]"


In [7]:
def escape(string):  # 正規表現の中でそのまま使いたい文字列をエスケープ
    res = !echo '{string}' | sed -e 's/[]\/$*.^[]/\\&/g'
    return res[0]

def escape_sed(string):  # sed の置換後の文字列用のエスケープ
    res = !echo '{string}' | sed -e 's/[\/&]/\\&/g'
    return res[0]

def run(cmd):
    p = subprocess.Popen(
        cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=isinstance(cmd, str)
    )
    for line in iter(p.stdout.readline, b''):
        print(line.decode("utf8"), end="")

def objective_with_index(trial, n_internal_parallel):
    index_parallel = f"{trial.number:04d}"
    print(f"{index_parallel=}")
    
    parameters_changed_filename = f"{index_parallel}_{FILENAME}"
    directory_input = Path("./in")  # 中のすべてのファイルに対して実行される
    
    run(["mkdir", f"{index_parallel}_out"])
    run(["mkdir", f"{index_parallel}_score"])
    
    # ファイル作成
    run(f"cp {FILENAME} {parameters_changed_filename}")
    sed_options = [f"-i {parameters_changed_filename}"]
    for variable in variables_optimize:
        if variable["method"] == "suggest_categorical":
            val =  trial.suggest_categorical(variable["name"], variable["choices"])
        else:
            val = getattr(trial, variable["method"])(variable["name"], variable["min"], variable["max"], log=variable["log"])
        left = variable["left"]
        right = variable["right"]
        sed_options.append(f"""-e 's/^{escape(left)}.*{escape(right)}$/{escape_sed(left)}{val}{escape_sed(right)}/'""")
    command_sed = f"sed {' '.join(sed_options)}"
    print(command_sed)
    run(command_sed)
    
    # コンパイル
    command_compile = f"clang++ {parameters_changed_filename} -std=gnu++17 -O2 -DONLINE_JUDGE -I/ -o {parameters_changed_filename}.out"
    print(command_compile)
    run(command_compile)
    # 実行・採点コマンド (@ はファイル名)
    command_exec = (
        #f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score"
        f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        #f"cargo run --release --manifest-path ../tools/Cargo.toml --bin tester {directory_input}/@ $(pwd)/{parameters_changed_filename}.out 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        #f"./{parameters_changed_filename}.out < {directory_input}/@ > ./{index_parallel}_out/@;"
        #f"cargo run --release --manifest-path ./tools/Cargo.toml --bin vis {directory_input}/@ ./{index_parallel}_out/@ 2> /dev/null > ./{index_parallel}_score/@;"
    )
    # 並列実行 (sed はパスのディレクトリ部分を消してファイル名にしてる)
    run(f"find {directory_input}/* | sed 's!^.*/!!' | xargs -I@ -P {n_internal_parallel} sh -c '{command_exec}'")
    
    # 集計
    sum_score = 0
    for file_path in Path(f"./{index_parallel}_score/").iterdir():  
        with open(file_path) as f:
            sum_score += int(f.read())
    
    # 後始末
    run(f"rm -rf {index_parallel}_out")
    run(f"rm -rf {index_parallel}_score")
    run(f"rm -rf {parameters_changed_filename}")
    run(f"rm -rf {parameters_changed_filename}.out")
    
    return sum_score


In [10]:
storage_path = f"study.db"
storage = f"sqlite:///{storage_path}"
study_name = "study"
study = optuna.create_study(storage=storage, load_if_exists=True, study_name=study_name, direction="maximize")

def callback(study, trial):
    try:
        if study.best_value == trial.value:
            print(f"Updated! {study.best_value}")
            index_parallel = f"{trial.number:04d}"
            parameters_changed_filename = f"{index_parallel}_{FILENAME}"
            !cp {parameters_changed_filename} 00000_best_parameters.cpp
        print("callback ok")
    except:
        print(":(")

[I 2021-11-12 21:29:17,631] A new study created in RDB with name: study


In [ ]:
objective = partial(objective_with_index, n_internal_parallel=8)
study.optimize(objective, n_trials=3000, timeout=86400, n_jobs=3, callbacks=[callback])

index_parallel='0001'
index_parallel='0000'
index_parallel='0002'


/home/user/anaconda3/lib/python3.8/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(


sed -i 0000_main.cpp -e 's/^constexpr auto MCMC_Q_L2_NORM_RANGE = .*;         \/\/ OPTIMIZE LOG \[2\.0, 20\.0\]$/constexpr auto MCMC_Q_L2_NORM_RANGE = 7.12038165153063;         \/\/ OPTIMIZE LOG [2.0, 20.0]/' -e 's/^constexpr auto MCMC_Q_RANGE = .*;                  \/\/ OPTIMIZE LOG \[0\.2, 10\.0\]$/constexpr auto MCMC_Q_RANGE = 6.3261068219554675;                  \/\/ OPTIMIZE LOG [0.2, 10.0]/' -e 's/^constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = .*; \/\/ OPTIMIZE LOG \[0\.02, 2\.0\]$/constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = 0.5606898684930279; \/\/ OPTIMIZE LOG [0.02, 2.0]/' -e 's/^constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = .*; \/\/ OPTIMIZE \[60, 100\]$/constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = 62; \/\/ OPTIMIZE [60, 100]/' -e 's/^constexpr auto PRIORITY_DAY_OFFSET = .*;          \/\/ OPTIMIZE \[400, 1200\]$/constexpr auto PRIORITY_DAY_OFFSET = 730;          \/\/ OPTIMIZE [400, 1200]/' -e 's/^constexpr auto PRIORITY_COEF = .*;               \/\/ OPTIMIZE LOG \[0\.0

[I 2021-11-12 21:33:13,891] Trial 0 finished with value: 1004564.0 and parameters: {'MCMC_Q_L2_NORM_RANGE': 7.12038165153063, 'MCMC_Q_RANGE': 6.3261068219554675, 'EXPECTED_SKILL_EMA_ALPHA_COEF': 0.5606898684930279, 'MAX_N_NOT_OPEN_TASKS_IN_QUEUE': 62, 'PRIORITY_DAY_OFFSET': 730, 'PRIORITY_COEF': 0.2161057863392588}. Best is trial 0 with value: 1004564.0.


Updated! 1004564.0
cp: cannot stat '0000_main.cpp': No such file or directory
callback ok
index_parallel='0003'
sed -i 0003_main.cpp -e 's/^constexpr auto MCMC_Q_L2_NORM_RANGE = .*;         \/\/ OPTIMIZE LOG \[2\.0, 20\.0\]$/constexpr auto MCMC_Q_L2_NORM_RANGE = 9.979906500688346;         \/\/ OPTIMIZE LOG [2.0, 20.0]/' -e 's/^constexpr auto MCMC_Q_RANGE = .*;                  \/\/ OPTIMIZE LOG \[0\.2, 10\.0\]$/constexpr auto MCMC_Q_RANGE = 4.011652596542475;                  \/\/ OPTIMIZE LOG [0.2, 10.0]/' -e 's/^constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = .*; \/\/ OPTIMIZE LOG \[0\.02, 2\.0\]$/constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = 0.6887869781397384; \/\/ OPTIMIZE LOG [0.02, 2.0]/' -e 's/^constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = .*; \/\/ OPTIMIZE \[60, 100\]$/constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = 80; \/\/ OPTIMIZE [60, 100]/' -e 's/^constexpr auto PRIORITY_DAY_OFFSET = .*;          \/\/ OPTIMIZE \[400, 1200\]$/constexpr auto PRIORITY_DAY_OFFSET = 1173;     

[I 2021-11-12 21:33:16,601] Trial 1 finished with value: 980024.0 and parameters: {'MCMC_Q_L2_NORM_RANGE': 10.678726965804834, 'MCMC_Q_RANGE': 8.501094578019957, 'EXPECTED_SKILL_EMA_ALPHA_COEF': 0.02598844115922582, 'MAX_N_NOT_OPEN_TASKS_IN_QUEUE': 97, 'PRIORITY_DAY_OFFSET': 435, 'PRIORITY_COEF': 0.013869754859411061}. Best is trial 0 with value: 1004564.0.


callback ok
index_parallel='0004'


[I 2021-11-12 21:33:16,992] Trial 2 finished with value: 999732.0 and parameters: {'MCMC_Q_L2_NORM_RANGE': 3.384117661909016, 'MCMC_Q_RANGE': 1.079497403481504, 'EXPECTED_SKILL_EMA_ALPHA_COEF': 0.10774228630014906, 'MAX_N_NOT_OPEN_TASKS_IN_QUEUE': 68, 'PRIORITY_DAY_OFFSET': 752, 'PRIORITY_COEF': 0.022027305020456505}. Best is trial 0 with value: 1004564.0.


callback ok
index_parallel='0005'
sed -i 0004_main.cpp -e 's/^constexpr auto MCMC_Q_L2_NORM_RANGE = .*;         \/\/ OPTIMIZE LOG \[2\.0, 20\.0\]$/constexpr auto MCMC_Q_L2_NORM_RANGE = 4.07884124658451;         \/\/ OPTIMIZE LOG [2.0, 20.0]/' -e 's/^constexpr auto MCMC_Q_RANGE = .*;                  \/\/ OPTIMIZE LOG \[0\.2, 10\.0\]$/constexpr auto MCMC_Q_RANGE = 0.6357423953046792;                  \/\/ OPTIMIZE LOG [0.2, 10.0]/' -e 's/^constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = .*; \/\/ OPTIMIZE LOG \[0\.02, 2\.0\]$/constexpr auto EXPECTED_SKILL_EMA_ALPHA_COEF = 0.027289589241608077; \/\/ OPTIMIZE LOG [0.02, 2.0]/' -e 's/^constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = .*; \/\/ OPTIMIZE \[60, 100\]$/constexpr auto MAX_N_NOT_OPEN_TASKS_IN_QUEUE = 66; \/\/ OPTIMIZE [60, 100]/' -e 's/^constexpr auto PRIORITY_DAY_OFFSET = .*;          \/\/ OPTIMIZE \[400, 1200\]$/constexpr auto PRIORITY_DAY_OFFSET = 771;          \/\/ OPTIMIZE [400, 1200]/' -e 's/^constexpr auto PRIORITY_COEF = .*;   

In [8]:
!rm -rf ./*_score
!rm -rf ./*_out
!rm -rf ./*_main.cpp*
!rm study.db